In [140]:
import numpy as np
import copy
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from sklearn.neural_network import MLPRegressor

## utils

In [141]:
def rmse(matrix1, matrix2):
    # 确保两个矩阵具有相同的维度
    if matrix1.shape != matrix2.shape:
        raise ValueError("Both matrices must have the same dimensions.")
    
    # 计算差异
    diff = matrix1 - matrix2
    
    # 计算均方误差（MSE）
    mse = torch.mean(diff ** 2)
    
    # 计算RMSE
    rmse = torch.sqrt(mse)
    
    return rmse.item()

def param_diff_loss(model, lambda_diff):
    # 提取各个逻辑回归层的权重
    w1 = model.log1.weight
    w2 = model.log2.weight
    w3 = model.log3.weight

    # 计算权重之间的绝对差值
    diff12 = torch.abs(w1 - w2).sum()
    diff23 = torch.abs(w2 - w3).sum()
    diff31 = torch.abs(w3 - w1).sum()

    # 返回加权的参数差异损失
    return lambda_diff * (diff12 + diff23 + diff31)

def gating_diff_loss(weights, lambda_gate):
    # 计算权重差异的绝对值
    diff = torch.abs(torch.max(weights, dim=1)[0]-torch.min(weights, dim=1)[0]).sum()
    # diff = (torch.abs(weights[:, :-1] - weights[:, 1:]).sum()+torch.abs(weights[:, -1] - weights[:, 0]).sum())
    # 返回加权的差异损失
    return -lambda_gate * diff/(weights.shape[0])


class GatingNetwork(nn.Module):
    def __init__(self, input_size, hidden_size, num_experts, dim_y):
        super(GatingNetwork, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, 2*hidden_size)
        self.fc3 = nn.Linear(2*hidden_size, num_experts)

        # 定义三个独立的逻辑回归模型作为专家
        self.log1 = nn.Linear(input_size, dim_y)
        self.log2 = nn.Linear(input_size, dim_y)
        self.log3 = nn.Linear(input_size, dim_y)

    def forward(self, x):
        # 门控网络部分
        x1 = F.relu(self.fc1(x))
        x1 = F.relu(self.fc2(x1))
        weights = F.softmax(self.fc3(x1), dim=1)  # 得到每个专家的权重

        # 专家的输出
        y1 = torch.sigmoid(self.log1(x)) * weights[:, 0].unsqueeze(1)  # 维度对齐
        y2 = torch.sigmoid(self.log2(x)) * weights[:, 1].unsqueeze(1)
        y3 = torch.sigmoid(self.log3(x)) * weights[:, 2].unsqueeze(1)

        # 合并各专家的加权输出
        return y1 + y2 + y3,weights
    

def train(model, optimizer, x_train, y_train, theta,tolerance=1e-4, max_epochs=1000,lam1=1,lam2=1):
    x_train = Variable(x_train, requires_grad=False)
    y_train = Variable(y_train, requires_grad=False)

    # last_loss = float('inf')
    for epoch in range(max_epochs):
        # 存储参数更新前的副本
        param_copies = {name: p.clone().detach() for name, p in model.named_parameters()}
        
        optimizer[0].zero_grad()
        outputs,weights = model(x_train)
        
        # loss = criterion(outputs, y_train)+param_diff_loss(model,lam1)
        loss = ((outputs-y_train)**2).sum()/(y_train.shape[0]*y_train.shape[1])+param_diff_loss(model,lam1)
        # a = nn.CrossEntropyLoss()
        # loss =  F.cross_entropy(outputs,y_train, reduction='sum')+param_diff_loss(model,lam1)
        loss.backward()
        optimizer[0].step()
        
        optimizer[1].zero_grad()
        outputs,weights = model(x_train)
        loss1 =  ((outputs-y_train)**2).sum()/(y_train.shape[0]*y_train.shape[1])+gating_diff_loss(weights, lam2)
        
        loss1.backward()
        optimizer[1].step()

        # 计算参数变化
        total_change = sum((p - param_copies[name]).abs().sum().item() for name, p in model.named_parameters())
        
        if (epoch+1)%100 == 0:
            print(f'Epoch {epoch+1}, Loss: {(loss+loss1).item()}, Max param change: {total_change} rmse:{rmse(model(x_train)[0],theta)}')
            # print(gating_diff_loss(weights, lam2),weights.shape)
        
        if total_change < tolerance:
            print("Training converged.")
            break
def px(x):
    p = x@torch.inverse(x.T@x)@x.T
    return torch.eye(p.shape[0],p.shape[1])-p



def step2(x,theta,y,M,lam1=1,lam2=1,a=1):
    beta = torch.inverse(x.T@x+lam1*torch.eye(x.shape[1],x.shape[1]))@x.T@(torch.mul(torch.mul(M,theta),y))
    # b1 = 1/(1+2*(y.shape[0]*y.shape[1]*lam2/2)*(1-a))
    # b2 = px(torch.mul(torch.mul(M,theta),y))
    # a1,b,c = torch.linalg.svd(b2, full_matrices=False)
    # b = b-a*(y.shape[0]*y.shape[1])*lam2/2
    # for i in range(b.shape[0]):
    #     b[i] = b[i] if b[i].item()>0 else 0
    # b = torch.diag_embed(b, 0, -2, -1)[:a1.size(0), :c.size(1)]
    # b = a1@b@c*b1
    return beta
        

## 参数设置及模拟数据生成

In [142]:
# 混合模型数量
K = 3
# 样本量
m = 400
# 协变量特征纬度
n1 = 20
# 观测矩阵维度
n2 = 400

# 协变量矩阵的生成
x = torch.randn(m,n1)
# 模型参数生成
w = torch.normal(0,0.2,(2,K,n1,n2))
# w[1,1,:,:] = torch.normal(0,1,(n1,n2))
# w[1,2,:,:] = torch.normal(0,2,(n1,n2))
b = torch.randn(2,K,n2)*0.1
#观测矩阵非缺失概率计算
theta = torch.zeros((m,n2))
y = torch.zeros((m,n2))
for i in range(m):
    k = np.random.choice(list(range(K)))
    theta[i,:] = torch.sigmoid(x[i,:]@w[0,k,:,:]+b[0,k,:])
    y[i,:]= x[i,:]@(w[1,k,:,:])
    
noise = torch.normal(0,((y-torch.mean(y))**2).sum()/(m*n2-1),(m,n2))
#根据概率生成对应的示性矩阵
M = torch.zeros((m,n2))
for i in range(m):
    for j in range(n2):
        M[i,j] = 1 if np.random.uniform(0,1) <= theta[i,j] else 0

## Step1: Estimation of $\hat{\theta}$ by mixed logistic regression

In [143]:
gating_network = GatingNetwork(n1,10,3,n2)
params_to_update = list(gating_network.log1.parameters()) + \
                   list(gating_network.log2.parameters()) + \
                   list(gating_network.log3.parameters())
optimizer = torch.optim.Adam(params_to_update, lr=0.1)

params_to_update1 = list(gating_network.fc1.parameters()) + \
                   list(gating_network.fc2.parameters()) + \
                   list(gating_network.fc3.parameters())
optimizer1 = torch.optim.Adam(params_to_update1, lr=0.1)
train(gating_network, [optimizer,optimizer1], x, M,theta=theta,tolerance=1e-3, max_epochs=1000,lam1=0.1,lam2=0.00001)


Epoch 100, Loss: 36.1435661315918, Max param change: 184.3721683472395 rmse:0.1922408938407898
Epoch 200, Loss: 33.885005950927734, Max param change: 178.98384596034884 rmse:0.1920389086008072
Epoch 300, Loss: 33.303314208984375, Max param change: 175.33259946852922 rmse:0.19167011976242065
Epoch 400, Loss: 33.21985626220703, Max param change: 175.41172036901116 rmse:0.19155991077423096
Epoch 500, Loss: 33.067298889160156, Max param change: 170.57601726986468 rmse:0.19191469252109528
Epoch 600, Loss: 33.20786666870117, Max param change: 176.42404648289084 rmse:0.19159920513629913
Epoch 700, Loss: 33.40910720825195, Max param change: 195.41078273952007 rmse:0.19080086052417755
Epoch 800, Loss: 32.989219665527344, Max param change: 169.85250714607537 rmse:0.19062930345535278
Epoch 900, Loss: 32.65434646606445, Max param change: 172.6575357913971 rmse:0.19100646674633026
Epoch 1000, Loss: 32.83058166503906, Max param change: 172.25990392267704 rmse:0.1909397542476654


## Step2

In [158]:
weight = gating_network(x)[1]
theta_hat = gating_network(x)[0]
# for i in range(m):
#     for j in range(n2):
#         if theta_hat[i,j] <0.1:
#             theta_hat[i,j] =0.1
index = torch.argmin(weight,axis=1)
y_hat = torch.zeros(m,n2)
for i in range(max(index)+1):
    data_index = np.where(index==i)[0].tolist()
    x_part = x[data_index]
    y_part = torch.mul(y,M)[data_index]
    M_part = M[data_index]
    theta_hat_part = 1/theta_hat[data_index]
    beta = step2(x_part,theta_hat_part,y_part+noise[data_index],M_part,10)
    y_hat[data_index] = x_part@beta
print(rmse(y_hat,y))
beta = step2(x,1/theta_hat,torch.mul(y+noise,M),M,10)
print(rmse(x@beta,y))

diff = torch.max(weight,axis=1)[0]
for i in range(m):
    y_hat[i] = (diff[i])*y_hat[i]+(1-diff[i])*(x@beta)[i]
print(rmse(y_hat,y))

0.8316338062286377
0.7976422905921936
0.7974525094032288


In [145]:
theta_hat

tensor([[0.4893, 0.5281, 0.5080,  ..., 0.4253, 0.5064, 0.5725],
        [0.5767, 0.4663, 0.4610,  ..., 0.3728, 0.5987, 0.5328],
        [0.6094, 0.5146, 0.3942,  ..., 0.4020, 0.5578, 0.4413],
        ...,
        [0.4065, 0.4541, 0.5654,  ..., 0.5601, 0.5444, 0.5375],
        [0.5133, 0.5384, 0.4962,  ..., 0.4790, 0.5559, 0.5641],
        [0.5574, 0.4775, 0.5781,  ..., 0.3848, 0.5468, 0.5708]],
       grad_fn=<AddBackward0>)